---
format: 
  html:
    toc: true
execute:
  echo: true
---

### Data Pre-processing

Let's import all the necessary libraries.

In [13]:
#| code-fold: true

# Basics
import numpy as np
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile # For opening .csv files inside the .zip files.
from itertools import chain # For flattening property and violent crime codes from the nested dictionary.
import os
from shapely.geometry import Point

#### Now, we can begin prepare the data. The first step is recovering the original data frames from the split files. The original .csv files are too large, so they are pre-split to facilitate uploading. The following codes are used to combine them back to their original forms.

In [2]:
def load_split_files(directory):
    all_parts = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith('.csv'):
            part_df = pd.read_csv(os.path.join(directory, filename))
            all_parts.append(part_df)
    return pd.concat(all_parts, ignore_index = True)

# Load the files
df_2010_2019 = load_split_files('./data/split_files_2010_2019')
df_2020_present = load_split_files('./data/split_files_2020_present')

**Note:** Both data sets contain more than 1 million rows.

In [15]:
print(f"The crime data from 2010 to 2019 contains {len(df_2010_2019)} rows.")
print(f"The crime data from 2010 to 2019 contains {len(df_2020_present)} rows.")

The crime data from 2010 to 2019 contains 1688586 rows.
The crime data from 2010 to 2019 contains 1001112 rows.


There is a slight caveat: For the crime data from 2010 to 2019, the variable name for the area code is "AREA " (note the trailing space), but for the crime data starting from 2020, the variable name for the area code is "AREA". So, I rename the column "AREA " to "AREA" in "df_2010_2019" so that both data frames are consistent in terms of variable names.

There are also repeated records, and these are dropped before merging.

In [3]:
df_2010_2019.rename(columns = {"AREA ": "AREA"}, inplace=True)
df_2010_2019 = df_2010_2019.drop_duplicates(subset = ["DR_NO"], keep = "first")

#### Now, I merge the two data frames together.

In [4]:
#| code-fold: true

data = pd.concat([df_2010_2019, df_2020_present]).reset_index(drop = True)

The descriptions of some of the variables in the data set are provided below.

#### Variables Description

- DATE OCC: Presumably date occured. In MM/DD/YYYY format.
- AREA: The LAPD has 21 Community Police Stations referred to as Geographic Areas within the department. These Geographic Areas are sequentially numbered from 1-21.
- Crm Cd: Indicates the crime committed. (Same as Crime Code 1)
- Crm Cd Desc: Defines the Crime Code provided.
- LAT: Latitude
- LON: Longtitude

These will be the variables to be used in the remaining of the analyses.

In [5]:
# Extract the relevant columns.

LAcrime = data[["DATE OCC", "AREA", "Crm Cd", "Crm Cd Desc", "LAT", "LON"]].copy()
LAcrime.head()

,DATE OCC,AREA,Crm Cd,Crm Cd Desc,LAT,LON
0,02/20/2010 12:00:00 AM,13,900,VIOLATION OF COURT ORDER,33.9825,-118.2695
1,09/12/2010 12:00:00 AM,14,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",33.9599,-118.3962
2,08/09/2010 12:00:00 AM,13,946,OTHER MISCELLANEOUS CRIME,34.0224,-118.2524
3,01/05/2010 12:00:00 AM,6,900,VIOLATION OF COURT ORDER,34.1016,-118.3295
4,01/02/2010 12:00:00 AM,1,122,"RAPE, ATTEMPTED",34.0387,-118.2488


Create a new "Date" column based on "DATE OCC", and then extract the year, month, and day, and store them in respective columns.

In [6]:
# Convert the dates stored in "DATE OCC" into a proper datetime format and store it as a new variable "Date".

LAcrime["Date"] = pd.to_datetime(LAcrime["DATE OCC"])
LAcrime["Year"] = LAcrime["Date"].dt.year
LAcrime["Month"] = LAcrime["Date"].dt.month
LAcrime["Day"] = LAcrime["Date"].dt.day_name()

LAcrime.head()

,DATE OCC,AREA,Crm Cd,Crm Cd Desc,LAT,LON,Date,Year,Month,Day
0,02/20/2010 12:00:00 AM,13,900,VIOLATION OF COURT ORDER,33.9825,-118.2695,2010-02-20,2010,2,Saturday
1,09/12/2010 12:00:00 AM,14,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",33.9599,-118.3962,2010-09-12,2010,9,Sunday
2,08/09/2010 12:00:00 AM,13,946,OTHER MISCELLANEOUS CRIME,34.0224,-118.2524,2010-08-09,2010,8,Monday
3,01/05/2010 12:00:00 AM,6,900,VIOLATION OF COURT ORDER,34.1016,-118.3295,2010-01-05,2010,1,Tuesday
4,01/02/2010 12:00:00 AM,1,122,"RAPE, ATTEMPTED",34.0387,-118.2488,2010-01-02,2010,1,Saturday


#### We focus on violent crimes and property crimes. The crime codes and categories are based on UCR Reporting.

In [7]:
# Nested dictionary of crime codes.

crime_codes = {
    "Violent": {
        "Homicide": [110, 113],
        "Rape": [121, 122, 815, 820, 821],
        "Robbery": [210, 220],
        "Aggravated Assaults": [230, 231, 235, 236, 250, 251, 761, 926],
        "Simple Assaults": [435, 436, 437, 622, 623, 624, 625, 626, 627, 647, 763, 928, 930]
    },
    "Property": {
        "Burglary": [310, 320],
        "Motor Vehicle Theft": [510, 520, 433],
        "Theft from Vehicle": [330, 331, 410, 420, 421],
        "Personal Theft": [350, 351, 352, 353, 450, 451, 452, 453],
        "Other Theft": [341, 343, 345, 440, 441, 442, 443, 444, 445, 470, 471, 472, 473, 474, 475, 480, 485, 487, 491]
    }
}

In [8]:
# Flatten the violent and property crime codes.
violent_codes = list(chain(*crime_codes["Violent"].values()))
property_codes = list(chain(*crime_codes["Property"].values()))

Based on the violent and property crime codes, create two new columns to store the corresponding crime categories ("Crime_Description") and crime type ("Crime Type").

In [9]:
# Flatten the crime_codes dictionary to map codes to descriptions.
code_to_description = {}
for crime_type, subdict in crime_codes.items():
    for description, codes in subdict.items():
        for code in codes:
            code_to_description[code] = description

# Add a "Crime_Description" column.
LAcrime["Crime_Description"] = LAcrime["Crm Cd"].map(code_to_description)

In [10]:
LAcrime = LAcrime[LAcrime["Crm Cd"].isin(violent_codes + property_codes)]

# Create conditions for assigning crime types.
conditions = [
    LAcrime["Crm Cd"].isin(violent_codes),
    LAcrime["Crm Cd"].isin(property_codes)
]

# Define corresponding choices (the crime types).
choices = ["violent", "property"]

# Assign the crime type based on the conditions.
LAcrime["Crime Type"] = np.select(conditions, choices, default = "other")

The pre-processed data frame looks like this:

In [11]:
LAcrime.head()

,DATE OCC,AREA,Crm Cd,Crm Cd Desc,LAT,LON,Date,Year,Month,Day,Crime_Description,Crime Type
4,01/02/2010 12:00:00 AM,1,122,"RAPE, ATTEMPTED",34.0387,-118.2488,2010-01-02,2010,1,Saturday,Rape,violent
5,01/04/2010 12:00:00 AM,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),34.0480,-118.2577,2010-01-04,2010,1,Monday,Other Theft,property
6,01/07/2010 12:00:00 AM,1,330,BURGLARY FROM VEHICLE,34.0389,-118.2643,2010-01-07,2010,1,Thursday,Theft from Vehicle,property
7,01/08/2010 12:00:00 AM,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",34.0435,-118.2427,2010-01-08,2010,1,Friday,Aggravated Assaults,violent
8,01/09/2010 12:00:00 AM,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",34.0450,-118.2640,2010-01-09,2010,1,Saturday,Aggravated Assaults,violent


#### Export the "LAcrime" dataframe as a compressed Python object. This will be used in further analyses.

In [12]:
LAcrime.to_parquet("LAcrime_trimmed", compression = "snappy")